In [ ]:
# Initialization

import math
from datetime import datetime
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import numpy as np
from IPython.core.display import HTML
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import plotly.graph_objects as go

import sklearn.metrics
import scipy.integrate as integrate
import scipy.optimize as sco

import fluids as fds
import ht 

import general as gen
import model as ty
import proc as pr
import heat_transfer as bht
import plot_functions as pfun
# importing sys
import sys
sys.path.insert(0,r'C:\Users\valen\Dropbox (DualSun)\PC\Documents\GitHub\RD-systems-and-test-benches')
import SK_standard as sk

import networkx as nx

In [ ]:
input_name = r'\Inputs_V4.41-NI-glass-backsheet.xlsx'

par = pr.import_input(r'G:\Drive partagés\BU04-Innovation\PVT-perf-1Dmodel\Inputs',input_name)
hyp = pr.create_dict_from_excel(r'G:\Drive partagés\BU04-Innovation\PVT-perf-1Dmodel\Hypotheses','\Model_hypotheses.xlsx','Main')
data = r'\230104_TUV_test_conditions_V4.41_dark_nowind.xlsx'
condi = pr.create_inputs_from_excel(data,par,hyp)

In [ ]:
# Parametric emissivity 
eps_list = [0.05,0.10,0.15,0.20,0.25,0.30,0.35]

param_res = {}

for i in range(len(eps_list)):
    par["exchanger"]["eps_he"] = eps_list[i]
    par["manifold"]["eps_he"] = eps_list[i]
    par["anomaly1"]["eps_he"] = eps_list[i]

    df_res,X,list_res,list_df,list_df_historic = ty.simu_condi(par,hyp,condi)
    param_res[eps_list[i]] = {'df_res':df_res,'X':X,'list_res':list_res,'list_df':list_df,'list_df_historic':list_df_historic}

In [ ]:
u_list = [0.3]

In [ ]:
[tab,X] = sk.find_SK_SST(condi,par['A_G'],columns_list=[0,1,0,0,0,0,0,0,0])
X_df = pd.DataFrame(X[0],index = ['eta0,hem','a1','a2','a3','a4','a5','a6','a7','a8'],columns=['TUV'])

for i in range(len(eps_list)):
    [tab_m,X_m] = sk.find_SK_SST(param_res[eps_list[i]]['df_res'],par['A_G'],columns_list=[0,1,0,0,0,0,0,0,0])
    X_df['Linear fit model 1D - eps = '+str(eps_list[i])] = X_m[0]

pr.disp_html(X_df)

In [ ]:
# X_df.to_csv('V4.41_glass_backsheet_dark_nowind_paramectric_study_emissivity.csv',sep=';')

In [ ]:
fig = go.Figure()

for i in range(len(u_list)):
    # Add traces

    for j in range(len(eps_list)):
        df_res = param_res[eps_list[j]]['df_res']

        fig.add_trace(go.Scatter(x=-df_res.loc[df_res["u"]==u_list[i]]['-(T_m - T_a)'], y=df_res.loc[df_res["u"]==u_list[i]]['Q_dot / A_G'],
                            mode = 'markers',
                            name='Model 1D - u = '+str(u_list[i])+' m/s' + ' - eps = '+str(eps_list[j]),
                            ))

        
        fig.add_trace(go.Scatter(x=-df_res.loc[df_res["u"]==u_list[i]]['-(T_m - T_a)'], y=sk.comp_power_rG_coeff_df(X_df['Linear fit model 1D - eps = '+str(eps_list[j])],df_res),
                            mode = 'lines',
                            line=dict(width=1,
                                dash='dashdot'),
                            name='Linear fit model 1D - u = '+str(u_list[i])+' m/s' + ' - eps = '+str(eps_list[j]),
                            ))
    
    fig.add_trace(go.Scatter(x=condi.loc[condi["u"]==u_list[i]]['T_m - T_a'], y=condi.loc[condi["u"]==u_list[i]]['Q_dot / A_G'],
                        mode = 'markers',
                        name='TUV - u = '+str(u_list[i])+' m/s')
                        )
    
    fig.add_trace(go.Scatter(x=condi.loc[condi["u"]==u_list[i]]['T_m - T_a'], y=sk.comp_power_rG_coeff_df(X_df['TUV'],condi),
                        mode = 'lines',
                        line=dict(color='red',width=1
                            ),
                        name='Linear fit TUV - u = '+str(u_list[i])+' m/s')
                        )
# Set x-axis title
fig.update_xaxes(title_text="T_m - T_amb")
fig.update_yaxes(title_text="Power related to gross (W/m2 coll.)")

fig.update_layout(
    title={
        'text': "V4.41 glass-backsheet dark no wind",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.update_layout(
    autosize=False,
    width=1200,
    height=700,
    margin=dict(
        l=0,
        r=0,
        b=50,
        t=50,
        pad=1
    ),
)

fig.show()

In [ ]:
fig.write_html('V4.41_glass_backsheet_dark_nowind_paramectric_study_emissivity.html')

In [ ]:
color_list = ['blue','red','green','orange','cyan','purple']

In [ ]:
pr.disp_html(df_res)

In [ ]:
pr.display_a_i(X)

In [ ]:
mse = sklearn.metrics.mean_squared_error(df_res['Q_dot / A_G'],condi['Q_dot / A_G'])
print(math.sqrt(mse))